확실한 spammer에 대한 정보를 가지기 위해서 (training에 사용할 데이터)

조건1 : reviewer Burstiness (RB) - focused on product (한 제품에 일정 날에 리뷰를 많이남긴 리뷰어)

조건2 : reviewer who has a lot of reviews (minimum리뷰수를 적절히 바꿔가면서 리뷰가 많은 리뷰어 찾기)

조건3 : 상호명(아뜨랑스, 고고싱, 스타일난다(난다))를 직접적으로 언급한 리뷰

조건4 : 평균 rScore가 5에 가까운 리뷰어

조건5 : 한 제품에 여러번 리뷰를 남긴 리뷰어

5개 모두 적용되는 리뷰어

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from konlpy.utils import pprint 

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:3: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:3: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: n

In [3]:
#gogosing
common_front = '../../dataset/go_0715/gogosing_' 
common_back = '.json'

def get_file(file_num):   
    file_path = common_front + str(file_num) + common_back
    json_data = open(file_path).read()
    data = json.loads(json_data)
    df_data = pd.DataFrame(data)
    return df_data

# 74번 파일이 에러가 나서 제외함
def concat_file(file_num):
    start_file = pd.DataFrame(get_file(1))
    for i in range(2,file_num+1):
        if( i == 74):
            continue
        df_tmp = pd.DataFrame(get_file(i))
        start_file = pd.concat([start_file, df_tmp])
    start_file = start_file.reset_index(drop=True)
    return start_file

go_data = concat_file(156)

In [4]:
print("total number of reviews in gogosing : " + str(go_data.shape[0]) )

total number of reviews in gogosing : 772829


# 네이버 페이 구매자 rScore 수정하기

In [7]:
naverpay = (go_data.loc[go_data['cId'] == u'네이버 페이 구매자'])
naverpay['rScore'].value_counts()

5.0    42029
3.0     9297
1.0     1213
Name: rScore, dtype: int64

In [8]:
for i in range(go_data.shape[0]):
    if(go_data.loc[i, 'cId'] == u'네이버 페이 구매자'):
        if(go_data.loc[i, 'desc'][0] == u'만'):
            go_data.loc[i, 'rScore'] = 5.0
        elif(naverpay.loc[i, 'desc'][0] == u'불'):
            go_data.loc[i, 'rScore'] = 1.0
        elif(naverpay.loc[i, 'desc'][0] == u'보'):
            go_data.loc[i, 'rScore'] = 3.0
    else:
        continue

# condition 1 : reviewer Burstiness (RB) - focused on product

(한 제품에 일정 날에 리뷰를 많이남긴 리뷰어)

In [21]:
def burst_in_product(min_count, dataframe):
    condition1 = []
    product_burst = pd.DataFrame({'count' : dataframe.groupby( [ "pID", "rDate", "cId"] ).size()}).reset_index()
    suspicious_reviewers = (product_burst[product_burst['count'] >= min_count])['cId']
    for reviewer in suspicious_reviewers:
        condition1.append(reviewer)
    return remove_duplicate_from_list(condition1)
    

In [25]:
def remove_duplicate_from_list(input_list):
    tmp_set = set(input_list)
    result = list(tmp_set)
    return result

# condition 2 : reviewer who has a lot of reviews

(minimum리뷰수를 적절히 바꿔가면서 리뷰가 많은 리뷰어 찾기)

In [47]:
def find_reviewer_who_has_many_reviews(min_review, dataframe):
    condition2 = []
    reviewer_and_reviews = dataframe['cId'].value_counts()
    suspicious_reviewers = reviewer_and_reviews[reviewer_and_reviews >= min_review].index
    for reviewer in suspicious_reviewers:
        condition2.append(reviewer)
    return condition2


# condition 3 : 상호명(아뜨랑스, 고고싱, 스타일난다(난다))를 직접적으로 언급한 리뷰어

In [48]:
def reviewer_who_directly_write_sitename(min_count, dataframe):
    condition3 = [] 
    reviewer_who_write_sitename = dataframe[dataframe['desc'].str.contains(u"고고싱")]
    reviewer_and_reviews = reviewer_who_write_sitename['cId'].value_counts()
    suspicious_reviewers = reviewer_and_reviews[reviewer_and_reviews >= min_count].index
    for reviewer in suspicious_reviewers:
        condition3.append(reviewer)
        
    return condition3

# condition4 : 평균 rScore가 5에 가까운 리뷰어

In [49]:
def reviewer_who_has_high_rscore(min_rscore, dataframe):
    condition4 = [] 
    mean_rscore = dataframe.groupby(dataframe.cId).mean()['rScore']
    suspicious_reviewers = mean_rscore[mean_rscore >= min_rscore].index
    for reviewer in suspicious_reviewers:
        condition4.append(reviewer)
        
    return condition4

In [50]:
def intersect(lst1, lst2):
    return list(set(lst1) & set(lst2))

#spam_reviewers = (intersect(intersect(intersect(condition3, condition4), condition2), condition1))
#pprint(spam_reviewers)

# condition 5 : reviewer who has many reviews in A product over all time

In [51]:
def reviewer_who_has_many_reviews_in_a_product(min_count, dataframe):
    condition5 = []
    product_burst = pd.DataFrame({'count' : dataframe.groupby( [ "pID", "cId"] ).size()}).reset_index()
    suspicious_reviewers = (product_burst[product_burst['count'] >= min_count])['cId']
    for reviewer in suspicious_reviewers:
        condition5.append(reviewer)
    
    return remove_duplicate_from_list(condition5)

#reviewer_who_has_many_reviews_in_a_product(7)

# condition 6 : reviewer who has many reviews in A day

In [52]:
def reviewer_who_has_many_reviews_in_a_day(min_count, dataframe):
    condition6 = []
    date_burst = pd.DataFrame({'count' : dataframe.groupby( [ "rDate", "cId"] ).size()}).reset_index()
    suspicious_reviewers = (date_burst[date_burst['count'] >= min_count])['cId']
    for reviewer in suspicious_reviewers:
        condition6.append(reviewer)
    
    return remove_duplicate_from_list(condition6)

# doc2vec

In [9]:
doc2vec_cid_df = pd.read_excel('./gogosing_doc2vec.xlsx', sheetname='Sheet1')

In [11]:
doc2vec_cid = doc2vec_cid_df[0].tolist()
len(doc2vec_cid)

7523

In [17]:
df = go_data.loc[go_data['cId']==doc2vec_cid[0]]

In [18]:
df

,_id,bodySize,cId,desc,height,helpful,pColor,pID,pSize,pTitle,pURL,rDate,rImages,rNo,rScore,weight
240119,gogosing_550609,55.0,sumin12**,어이가없네요.. 처음에 오자마자 입어봤는데 한쪽이 붕떠서 바지 길이재봤더니 길이가 ...,155.0,"[42, 64]",중청(blue),34760,M,여리발목 크롭팬츠#스판함유#슬림핏#일자팬츠,http://www.ggsing.com/product/detail.html?prod...,2017. 01. 18,[assets1.cre.ma/p/ggsing-com/reviews/00/00/55/...,550609,5.0,44~46
245486,gogosing_545125,44.0,sumin12**,정말 골지티 잘산것같어요 목부분도 목막히지도않고 쫀쫀하고 가격이착해서 조금 걱정했지...,1.0,"[0, 0]",블랙,34114,선택안함,꺼내입어요 골지폴라티,http://www.ggsing.com/product/detail.html?prod...,2017. 01. 03,NaN,545125,5.0,80kg이상
247625,gogosing_542935,44.0,sumin12**,골지티가 없어서 골지티 찾고 있었는데 마침 고고싱에 있길래 샀어요 생각보다 심하게 ...,1.0,"[0, 0]",화이트,34114,선택안함,꺼내입어요 골지폴라티,http://www.ggsing.com/product/detail.html?prod...,2016. 12. 27,NaN,542935,5.0,80kg이상
326548,gogosing_460923,44.0,sumin12**,생각보다 비침이 심하지 않아서 괜찮아요 ㅎㅎ 딱 적당한 비침이에요! 슬랙스랑 입으면...,1.0,"[0, 0]",화이트,31387,NaN,에블데이 슬라브티,http://www.ggsing.com/product/detail.html?prod...,2016. 06. 19,NaN,460923,5.0,80kg이상
326549,gogosing_460922,44.0,sumin12**,저는 중청샀는데 ㅎㅎ 바지 디자인이 너무 이뻐서 고민도 안하고 옷 주문햇는데 역시나...,1.0,"[0, 0]",중청(blue denim),31963,M,구제컷반하이 3부숏팬츠,http://www.ggsing.com/product/detail.html?prod...,2016. 06. 19,NaN,460922,5.0,80kg이상
326553,gogosing_460918,44.0,sumin12**,m 사니까 허리도 딱 맞고 길이도 적당하고 다 괜찮았어요 근데 하얀색이라 역시 비침...,1.0,"[0, 0]",화이트(White),31592,M,뽀샵없이 하이숏팬츠,http://www.ggsing.com/product/detail.html?prod...,2016. 06. 19,NaN,460918,5.0,80kg이상
326554,gogosing_460917,44.0,sumin12**,색깔이 생각보다 밝아요!사진과는 다른것같네요ㅠㅠ조금더 어두웠으면 했는데ㅠㅠ 그래도 ...,1.0,"[0, 0]",진청(Dark-Blue),31507,L,포토샵 하이숏팬츠 (데님),http://www.ggsing.com/product/detail.html?prod...,2016. 06. 19,NaN,460917,5.0,80kg이상
326555,gogosing_460916,44.0,sumin12**,소라색 추천 많아서 샀는데 생각보다 질도 너무 좋고 색도 너무 예뻐요~ 면이 좀 두...,1.0,"[0, 0]",소라(sky blue),32031,NaN,솜사탕9부티 반팔,http://www.ggsing.com/product/detail.html?prod...,2016. 06. 19,NaN,460916,5.0,80kg이상
326557,gogosing_460914,44.0,sumin12**,진짜 커요..완전완전 제가 가지고 있는 옷중에 제일로 커요 프린팅은 부들부들하고 짱...,1.0,"[0, 0]",NaN,31156,캠프,빈티지프린팅 반티,http://www.ggsing.com/product/detail.html?prod...,2016. 06. 19,NaN,460914,5.0,80kg이상
326559,gogosing_460912,44.0,sumin12**,사진보고 예뻐서 구매햇는데 막상 입어보니 조금커서 아쉬웠지만 ㅠㅠㅠㅠ그래도 나름대로...,1.0,"[0, 0]",블랙(BLACK),31448,NaN,썸머린넨 점프수트,http://www.ggsing.com/product/detail.html?prod...,2016. 06. 19,NaN,460912,5.0,80kg이상


In [20]:
len(df)

278

In [72]:
def find_which_features_included (dataframe, cond1, cond2, cond3, cond4, cond5, cond6):
    print("--------------------")
    
    condition1_num = 0
    condition2_num = 0
    condition3_num = 0
    condition4_num = 0
    condition5_num = 0
    condition6_num = 0

    reviewer1 = burst_in_product(cond1, dataframe)
    reviewer2 = find_reviewer_who_has_many_reviews(cond2, dataframe)
    reviewer3 = reviewer_who_directly_write_sitename(cond3, dataframe)
    reviewer4 = reviewer_who_has_high_rscore(cond4, dataframe)
    reviewer5 = reviewer_who_has_many_reviews_in_a_product(cond5, dataframe)
    reviewer6 = reviewer_who_has_many_reviews_in_a_day(cond6, dataframe)
    
    if(len(reviewer1)!=0):
        condition1_num+=1
        print("cond1 included")
    if(len(reviewer2)!=0):
        condition2_num= condition2_num + 1
        print("cond2 included")
    if(len(reviewer3)!=0):
        condition3_num+=1
        print("cond3 included")
    if(len(reviewer4)!=0):
        condition4_num+=1
        print("cond4 included")
    if(len(reviewer5)!=0):
        condition5_num+=1
        print("cond5 included")
    if(len(reviewer6)!=0):
        condition6_num+=1
        print("cond6 included")
        
    return condition1_num,condition2_num, condition3_num, condition4_num, condition5_num, condition6_num

In [73]:
find_which_features_included(df, 10, 100, 15, 4.98, 10, 15)

--------------------
cond2 included
cond3 included
cond4 included
cond6 included


(0, 1, 1, 1, 0, 1)

# reviewer_who_satisfy_some_condition - intersect ver.

In [31]:
intersect_public_total = []
intersect_public_rscore_5 = []
intersect_public_rscore_4 = []
intersect_public_rscore_3 = []
intersect_public_rscore_2 = []
intersect_public_rscore_1 = []

In [32]:
'''
cond1 : 한 제품 & 특정 날짜에 cond1개 초과로 남긴 리뷰어
cond2 : cond2개 초과의 리뷰를 가진 리뷰어
cond3 : 상호명 언급을 cond3번 초과한 리뷰어
cond4 : 평점 cond4초과 리뷰어
cond5 : 한제품에 cond5개 초과로 리뷰남긴 리뷰어 추가하기
'''
def reviewer_who_satisfy_some_condition(dataframe, cond1_bool, cond2_bool, cond3_bool, cond4_bool, cond5_bool, cond6_bool,
                                        cond1, cond2, cond3, cond4, cond5, cond6, max_rscore):
    
    unique_cid = dataframe.cId.unique()
    #condition1
    if(cond1_bool == 1):
        reviewer1 = burst_in_product(cond1, dataframe)
    else:
        reviewer1 = unique_cid
    
    #condition2
    if(cond2_bool == 1):
        reviewer2 = find_reviewer_who_has_many_reviews(cond2, dataframe)
    else:
        reviewer2 = unique_cid
        
    #condition3
    if(cond3_bool == 1):
        reviewer3 = reviewer_who_directly_write_sitename(cond3, dataframe)
    else:
        reviewer3 = unique_cid
    
    #condition4
    if(cond4_bool == 1):
        reviewer4 = reviewer_who_has_high_rscore(cond4, dataframe)
    else:
        reviewer4 = unique_cid
        
    #condition5
    if(cond5_bool == 1):
        reviewer5 = reviewer_who_has_many_reviews_in_a_product(cond5, dataframe)
    else:
        reviewer5 = unique_cid
        
     #condition6
    if(cond6_bool == 1):
        reviewer6 = reviewer_who_has_many_reviews_in_a_day(cond6, dataframe)
    else:
        reviewer6 = unique_cid
        
    spam_reviewers = intersect(intersect(intersect(intersect(intersect(reviewer1, reviewer2), reviewer3), reviewer4), reviewer5), reviewer6)
    
    #네이버 페이 구매자는 제외
    if(u'\ub124\uc774\ubc84 \ud398\uc774 \uad6c\ub9e4\uc790' in spam_reviewers):
        spam_reviewers.remove(u'\ub124\uc774\ubc84 \ud398\uc774 \uad6c\ub9e4\uc790')
    
    except_spam_in_gogosing = dataframe[~dataframe.cId.isin(spam_reviewers)]
    print("the number of spam_reviewers : " + str(len(spam_reviewers)))
    print("the number of non spam reviewers : " + str(len(except_spam_in_gogosing.cId.unique())))
    
    #제외될 리뷰중 3.0 이하의 리뷰는 제거하지 않음
    low_rscore_in_spam_reviewers = (dataframe[dataframe.cId.isin(spam_reviewers)]).loc[dataframe['rScore']<= max_rscore]
    except_spam_in_gogosing = pd.concat([except_spam_in_gogosing, low_rscore_in_spam_reviewers])
    
    print("<rScore distribution>")
    print("cond1 =" + str(cond1) + ", cond2 =" + str(cond2) + ", cond3 =" + str(cond3) + ", cond4 =" + str(cond4) + ", cond5 =" + str(cond5) + ", cond6 = " + str(cond6))
    rscore = except_spam_in_gogosing['rScore'].value_counts()
    intersect_public_total.append(except_spam_in_gogosing.shape[0])
    intersect_public_rscore_5.append(rscore[5.0])
    intersect_public_rscore_4.append(rscore[4.0])
    intersect_public_rscore_3.append(rscore[3.0])
    intersect_public_rscore_2.append(rscore[2.0])
    intersect_public_rscore_1.append(rscore[1.0])
    
    
    #print("except spam in gogosing : " )
    #print(except_spam_in_gogosing)

    
    if(not(5.0 in rscore.index)):
        add_row = pd.Series([1], index=[5.0])
        rscore = rscore.add(add_row)
    if(not(4.0 in rscore.index)):
        add_row = pd.Series([1], index=[4.0])
        rscore = rscore.add(add_row)
    if(not(3.0 in rscore.index)):
        add_row = pd.Series([1], index=[3.0])
        rscore = rscore.add(add_row)
    if(not(2.0 in rscore.index)):
        add_row = pd.Series([1], index=[2.0])
        rscore = rscore.add(add_row)
    if(not(1.0 in rscore.index)):
        add_row = pd.Series([1], index=[1.0])
        rscore = rscore.add(add_row)
        
    print("\n")
    print("rscore : ")
    print(rscore)        
    
    print ("ratio btw 5.0 and 4.0 : " + str(float(rscore[5.0])/float(rscore[4.0])))
    print ("ratio btw 5.0 and 3.0 : " + str(float(rscore[5.0])/float(rscore[3.0])))
    print ("ratio btw 5.0 and 2.0 : " + str(float(rscore[5.0])/float(rscore[2.0])))
    print ("ratio btw 5.0 and 1.0 : " + str(float(rscore[5.0])/float(rscore[1.0])))
    
    print ("the number of 5.0 reviews : " + str(rscore[5.0]))
    print ("the number of 4.0 reviews : " + str(rscore[4.0]))
    print ("the number of 3.0 reviews : " + str(rscore[3.0]))
    print ("the number of 2.0 reviews : " + str(rscore[2.0]))
    print ("the number of 1.0 reviews : " + str(rscore[1.0]))
       
    print(spam_reviewers)
        
    score = ('5.0', '4.0', '3.0', '2.0', '1.0')
    number_of_reviews = [rscore[5.0],
                     rscore[4.0],
                     rscore[3.0],
                     rscore[2.0],
                     rscore[1.0]]

    plt.bar(score, number_of_reviews, color=['red'],
        width=0.3, alpha=0.5)
    plt.xticks(score, fontsize=15)
    plt.yticks(fontsize=15)
    plt.ylabel('# of review', fontsize=15)
    plt.rcParams["figure.figsize"] = (6,4)
    plt.xlabel('rScore', fontsize=15)
    ax = plt.gca()
    ax.set_ylim([0.0, except_spam_in_gogosing.shape[0]+500])
    plt.show()
    print("\n\n")
    return spam_reviewers


In [ ]:
#3.0이하의 데이터도 걸려졌으면 삭제
for cond1 in range(3, 11, 2):
    for cond2 in range(50, 300, 50):
        for cond3 in range(10, 40, 5):
            for cond4 in range(1, 11, 2):
                for cond5 in range(2, 20, 3):
                    for cond6 in range(5, 15, 2):
                        rscore = cond4*0.01 + 4.9
                        reviewer_who_satisfy_some_condition(go_data, 1, 1, 1, 1, 1, 1, cond1, cond2, cond3, rscore, cond5, cond6, -1)

# reviewer_who_satisfy_some_condition - union ver.

In [14]:
union_public_total = []
union_public_rscore_5 = []
union_public_rscore_4 = []
union_public_rscore_3 = []
union_public_rscore_2 = []
union_public_rscore_1 = []

In [20]:
def reviewer_who_satisfy_some_condition_union(dataframe, cond1_bool, cond2_bool, cond3_bool, cond4_bool, cond5_bool, cond6_bool,
                                        cond1, cond2, cond3, cond4, cond5, cond6, max_rscore):
    
     #condition1
    if(cond1_bool == 1):
        reviewer1 = burst_in_product(cond1, dataframe)
    else:
        reviewer1 = []
    
    #condition2
    if(cond2_bool == 1):
        reviewer2 = find_reviewer_who_has_many_reviews(cond2, dataframe)
    else:
        reviewer2 = []
        
    #condition3
    if(cond3_bool == 1):
        reviewer3 = reviewer_who_directly_write_sitename(cond3, dataframe)
    else:
        reviewer3 = []
    
    #condition4
    if(cond4_bool == 1):
        reviewer4 = reviewer_who_has_high_rscore(cond4, dataframe)
    else:
        reviewer4 = []
        
    #condition5
    if(cond5_bool == 1):
        reviewer5 = reviewer_who_has_many_reviews_in_a_product(cond5, dataframe)
    else:
        reviewer5 = []  

    #condition6
    if(cond6_bool == 1):
        reviewer6 = reviewer_who_has_many_reviews_in_a_day(cond6, dataframe)
    else:
        reviewer6 = []
        
    spam_reviewers = remove_duplicate_from_list(reviewer1 + reviewer2 + reviewer3 + reviewer4 + reviewer5 + reviewer6)
    
    
    #네이버 페이 구매자는 스패머에서 일단 제외
    if(u'\ub124\uc774\ubc84 \ud398\uc774 \uad6c\ub9e4\uc790' in spam_reviewers):
        spam_reviewers.remove(u'\ub124\uc774\ubc84 \ud398\uc774 \uad6c\ub9e4\uc790')
    
    except_spam_in_gogosing = dataframe[~dataframe.cId.isin(spam_reviewers)]
    print("total number of expected non-spam reviewers : " + str(len(except_spam_in_gogosing.cId.unique()))) 
    
    #제외될 리뷰중 3.0 이하의 리뷰는 제거하지 않음
    low_rscore_in_spam_reviewers = (dataframe[dataframe.cId.isin(spam_reviewers)]).loc[dataframe['rScore']<= max_rscore]
    except_spam_in_gogosing = pd.concat([except_spam_in_gogosing, low_rscore_in_spam_reviewers])
    
    print("total number of suspicious spam reviewers : " + str(len(spam_reviewers)))
      
    print("<rScore distribution>")
    print("cond1 =" + str(cond1) + ", cond2 =" + str(cond2) + ", cond3 =" + str(cond3) + ", cond4 =" + str(cond4) + ", cond5 =" + str(cond5) + ", cond6 =" + str(cond6)) 
    
    rscore = except_spam_in_gogosing['rScore'].value_counts()

    if(not(5.0 in rscore.index)):
        add_row = pd.Series([1], index=[5.0])
        rscore = rscore.add(add_row)
    if(not(4.0 in rscore.index)):
        add_row = pd.Series([1], index=[4.0])
        rscore = rscore.add(add_row)
    if(not(3.0 in rscore.index)):
        add_row = pd.Series([1], index=[3.0])
        rscore = rscore.add(add_row)
    if(not(2.0 in rscore.index)):
        add_row = pd.Series([1], index=[2.0])
        rscore = rscore.add(add_row)
    if(not(1.0 in rscore.index)):
        add_row = pd.Series([1], index=[1.0])
        rscore = rscore.add(add_row)  
        
    union_public_total.append(except_spam_in_gogosing.shape[0])
    union_public_rscore_5.append(rscore[5.0])
    union_public_rscore_4.append(rscore[4.0])
    union_public_rscore_3.append(rscore[3.0])
    union_public_rscore_2.append(rscore[2.0])
    union_public_rscore_1.append(rscore[1.0])

    non_spam_reviewers = except_spam_in_gogosing.cId.unique()
    non_spam_reviewers = non_spam_reviewers.tolist()
    
    if(u'\ub124\uc774\ubc84 \ud398\uc774 \uad6c\ub9e4\uc790' in non_spam_reviewers):
        non_spam_reviewers.remove(u'\ub124\uc774\ubc84 \ud398\uc774 \uad6c\ub9e4\uc790')
   
    print ("ratio btw 5.0 and 4.0 : " + str(float(rscore[5.0])/float(rscore[4.0])))
    print ("ratio btw 5.0 and 3.0 : " + str(float(rscore[5.0])/float(rscore[3.0])))
    print ("ratio btw 5.0 and 2.0 : " + str(float(rscore[5.0])/float(rscore[2.0])))
    print ("ratio btw 5.0 and 1.0 : " + str(float(rscore[5.0])/float(rscore[1.0])))
    

    print ("the number of 5.0 reviews : " + str(rscore[5.0]))
    print ("the number of 4.0 reviews : " + str(rscore[4.0]))
    print ("the number of 3.0 reviews : " + str(rscore[3.0]))
    print ("the number of 2.0 reviews : " + str(rscore[2.0]))
    print ("the number of 1.0 reviews : " + str(rscore[1.0]))
    
                    
    score = ('5.0', '4.0', '3.0', '2.0', '1.0')
    number_of_reviews = [rscore[5.0],
                     rscore[4.0],
                     rscore[3.0],
                     rscore[2.0],
                     rscore[1.0]]

    plt.bar(score, number_of_reviews, color=['red'],
        width=0.3, alpha=0.5)
    plt.xticks(score, fontsize=15)
    plt.yticks(fontsize=15)
    plt.ylabel('# of review', fontsize=15)
    plt.rcParams["figure.figsize"] = (6,4)
    plt.xlabel('rScore', fontsize=15)
    ax = plt.gca()
    ax.set_ylim([0.0, except_spam_in_gogosing.shape[0]+1000])
    plt.show()
    print("\n\n")
    return non_spam_reviewers